## **Outil d'alignement des données appliqué à des artistes sculpteurs**

Cet outil a été réalisé dans le cadre du projet MonumenTAL. Il permet de récolter des informations sur des personnages en allant questionner la base de connaissances Wikidata. Pour cet exemple, il s'agit d'artistes sculpteurs français.

*Document d'entrée* : un fichier Excel contenant la liste des artistes. /!\ Attention : les noms et prénoms des artistes doivent se trouver dans la première colonne de la première feuille du classeur.

*Documents de sortie* : 3 fichiers, un contenant la liste des identifiants Wikidata des artistes recherchés (.txt), un autre contenant les informations sur les artistes (.xlsx) et un dernier contenant ces mêmes informations dans un format pour une réutilisation machine (.json).

In [ ]:
#@markdown # Connexion du notebook à son compte Google Drive et signalement du dossier de travail :

#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

''' 
Google Colab notebook.
Python == 3.7.11

BaOIA - La Contemporaine - Université de Nanterre
'''

## Installation des bibliothèques et connexion au compte Google Drive
from google.colab import drive
import re
import glob
import os
import itertools
import json
import pandas as pd
import urllib.parse, urllib.request, json
import collections
from collections import Counter
!pip install wptools==0.4.17
import wptools

drive.mount('/content/drive/')
%cd /content/drive/My Drive/

#@markdown ###Indiquer le chemin vers le dossier de travail sur le Google Drive (si le dossier n'existe pas, il sera créé lors du lancement de la cellule) :
chemin_vers_dossier_travail = '/content/drive/My Drive/artistes_sculpteurs/'#@param {type:"string"}

if not os.path.exists(chemin_vers_dossier_travail):
      os.makedirs(chemin_vers_dossier_travail)
os.chdir(chemin_vers_dossier_travail)

In [ ]:
#@markdown # Récupération des informations Wikidata des artistes contenus dans le fichier Excel :

# Importation du fichier excel contenant la liste des artistes
#@markdown ####Indiquer le chemin vers le fichier Excel sur le Google Drive contenant la liste des artistes :
chemin_vers_fichier_excel = '/content/drive/My Drive/artistes_sculpteurs/artistes_francais.xlsx' #@param {type:"string"}
export = pd.read_excel(chemin_vers_fichier_excel)
liste_artistes = export.values.T[0].tolist()
print(liste_artistes)

#@markdown ####Changer la langue si nécessaire (par défaut : français) :
langue = "fr" #@param {type:"string"}
#@markdown ####Si la détection des personnes n'est pas assez précise, descendre le seuil à 0.8 (moins de noms vont être identifiés). Dans le cas contraire, monter le seuil à 1.0 :
seuil = "1.0" #@param {type:"string"}

def CallWikifier(text, lang=langue, threshold=float(seuil)): 

  """ FROM: http://wikifier.org/info.html

  Fonction qui associe chaque entitée nommée reconnue dans un texte à un identifiant
  unique se rapportant à la base de donnée Wikidata. Cela permet d'identifier et de
  désambiguiser les noms de personnages, de lieux, d'institutions. 
  
  La fonction prend en entrée le texte (les noms, lieux etc.) ainsi que la langue
  qu'il est possible de modifier ('en' pour anglais, 'fr' pour français) 
  
  Il est possible de changer le seuil 'threshold' (entre 0.8 et 1.0): plus le taux est élevé,
  plus de résultats seront repérés mais plus d'erreurs apparaissent. """

  data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "4"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
  url = "http://www.wikifier.org/annotate-article"
  req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
  with urllib.request.urlopen(req, timeout = 500) as f:
      global response
      response = f.read()
      response = json.loads(response.decode("utf8"))

# Création des listes et dictionnaires qui vont contenir les informations 
# récupérées grâce à l'identifiant unique Wikidata.
donnees_artistes_monumental = {}
nom = []
annee_naissance_search = []
date_naissance_data = []
date_deces_data = []
lieu_naissance_data = []
annee_deces_search = []
lieu_deces_data = []
lieu_naissance_net = []
sexe = []
lieu_deces_net = []
liste_donnees = []
liste_perso_corpus = []
itemid = []

# Pour chaque artiste, appel de la fonction Callwikifier 
# Récupération des informations: date de naissance, date de décès,
# lieu de naissance, lieu de décès, genre et enregistrement sous
# forme de dictionnaire

for artiste in liste_artistes:
  CallWikifier(text=artiste)
  try:
    for annotation in response["annotations"]: ## pour chaque réponse:
      itemid = annotation["wikiDataItemId"]
      page = wptools.page(wikibase=itemid, lang="fr")
      page.get_wikidata()
      page.get_parse()
      infobox=page.data['infobox']
      if infobox is not None:
        nom = infobox.get("nom")
        print(nom)
        tout=page.data['wikidata']
        sexe=tout.get("sexe ou genre (P21)")
        date_naissance=infobox.get("date de naissance")
        annee_naissance_search = re.findall(r'\b\d{3,4}\b', str(date_naissance)) ## Conserver uniquement les années
        date_deces=infobox.get("date de décès")
        annee_deces_search = re.findall(r'\b\d{3,4}\b', str(date_deces)) ### Conserver uniquement les années
        lieu_naissance = infobox.get("lieu de naissance") 
        lieu_naissance2 = re.sub(r'([()[\]{}]|)', "",str(lieu_naissance))
        lieu_naissance3 = re.sub(r'(<br>)(<br\>)', "",str(lieu_naissance2))
        lieu_naissance_net = re.sub(r'({{-}})', "",str(lieu_naissance3))
        lieu_deces=infobox.get("lieu de décès")
        lieu_deces2 = re.sub(r'([()[\]{}]|)', "",str(lieu_deces))
        lieu_deces3 = re.sub(r'(<br>)(<br\>)', "",str(lieu_deces2))
        lieu_deces_net = re.sub(r'({{-}})', "",str(lieu_deces3))
        donnees_artistes_monumental[nom] = {"Date de naissance" : annee_naissance_search, "Date de deces" : annee_deces_search, 
             "Lieu de naissance" : lieu_naissance_net, "Lieu de deces" : lieu_deces_net, "Genre": sexe}
  except (TypeError, ValueError, LookupError) as tpe:
    pass 

#@markdown ####Indiquer le noms des trois fichiers obtenus :
nom_fichier_liste_des_identifiants_wikidata = 'liste_identifiants_artistes_francais.txt' #@param {type:"string"}
with open(nom_fichier_liste_des_identifiants_wikidata, "w") as koui:
    koui.write(str(itemid))

nom_fichier_donnees_artistes = 'donnees_artistes_francais.json' #@param {type:"string"}
with open(nom_fichier_donnees_artistes, "w") as gjoi:
  json.dump(donnees_artistes_monumental, gjoi, indent=4, ensure_ascii=False)

nom_fichier_excel_donnees_artistes = 'informations_artistes.xlsx' #@param {type:"string"}
df = pd.DataFrame(donnees_artistes_monumental)
df2 = df.transpose()
df2.to_excel(nom_fichier_excel_donnees_artistes)